In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
# check folder status
folderLst = ['mdl', 'tab']
for folder in folderLst:
    if not os.path.exists('../' + folder):
        os.makedirs('../' + folder)

In [3]:
# load data without review; parse date
dat = pd.read_csv('../dat/yelp_academic_dataset_review_nr.csv', parse_dates=['date'])
print len(dat)

4153150


In [4]:
# load business data
meta = pd.read_csv('../dat/yelp_academic_dataset_business.csv')

In [5]:
# remain only 'Restaurants'
print len(dat.business_id.unique())
restaurant_id = set([i for i, c in meta.loc[pd.notnull(meta.categories),['business_id', 'categories']].values
                    if 'Restaurants' in c])
print len(restaurant_id)

144072
48485


In [6]:
dat = dat[dat.business_id.isin(restaurant_id)]
print len(dat)

2577298


In [7]:
# filter based on item counts
ITEM_THRES = 10

item_count = dat.business_id.value_counts()
print 'number of items: %d' % len(item_count)

dat = dat.set_index('business_id', drop=False)
dat['item_count'] = item_count
dat = dat.loc[dat['item_count'] >= ITEM_THRES]

number of items: 48485


In [8]:
# filter based on user counts
USER_THRES = 5

user_count = dat.user_id.value_counts()
print 'number of users: %d' % len(user_count)

dat = dat.set_index('user_id', drop=False)
dat['user_count'] = user_count
dat = dat.loc[dat['user_count'] >= USER_THRES]

number of users: 707751


In [9]:
# consider unique items and users
uni_item = dat.business_id.unique()
NI = len(uni_item)
uni_user = dat.user_id.unique()
NU = len(uni_user)
print 'number of items: %d' % NI
print 'number of users: %d' % NU

number of items: 31792
number of users: 109160


In [10]:
# write lists of users
with open('../tab/userList', 'w') as f:
    f.write(str(len(uni_user)) + '\n')
    f.write('\n'.join(uni_user))

# write lists of items
with open('../tab/itemList', 'w') as f:
    f.write(str(len(uni_item)) + '\n')
    f.write('\n'.join(uni_item))

In [11]:
# create mapping for item
item_map = dict()
for idx, item_id in enumerate(uni_item):
    item_map[item_id] = idx
    
# create mapping for user
user_map = dict()
for idx, user_id in enumerate(uni_user):
    user_map[user_id] = idx

In [12]:
# create dataset in tuple
tn = np.array([(user_map[u], item_map[i], r, t.year, t.month, t.day, t.dayofweek) \
               for u, i, r, t in dat.loc[:,['user_id', 'business_id', 'stars', 'date']].values \
               if t.year <= 2015])
tt = np.array([(user_map[u], item_map[i], r, t.year, t.month, t.day, t.dayofweek) \
               for u, i, r, t in dat.loc[:,['user_id', 'business_id', 'stars', 'date']].values \
               if t.year == 2016])

In [13]:
# write tn as tuple
with open('../tab/tnList', 'w') as f:
    f.write(str(tn.shape[0]) + '\n')
    f.write('\n'.join([','.join(map(str,e)) for e in tn]))
    
# write tn as tuple
with open('../tab/ttList', 'w') as f:
    f.write(str(tt.shape[0]) + '\n')
    f.write('\n'.join([','.join(map(str,e)) for e in tt]))

In [14]:
print tn.shape
print tt.shape

(1206403, 7)
(325761, 7)
